In [ ]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.cluster import OPTICS, cluster_optics_dbscan
import matplotlib.gridspec as gridspec
from sklearn.cluster import KMeans

In [ ]:
def create_dataset(img_folder):
    img_data_array=[]
    for file in os.listdir(os.path.join(img_folder)):
            image_path= os.path.join(img_folder,  file)
            image= cv2.imread( image_path, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, (270, 480))
            img_data_array.append(image)
    return img_data_array

In [ ]:
img_data = create_dataset('database')
print(len(img_data))
print(img_data[0].shape)

In [ ]:
listKp = []
listDp = []
listDpNp = np.array([])
listHist = []
listFFT = []
bins=(8, 8, 8)
for idx, image in enumerate(img_data):
    f = np.fft.fft2(image)
    fshift = np.fft.fftshift(f)
    magnitude_spectrum = np.abs(fshift)
    listFFT.append(magnitude_spectrum)
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv], [0, 1, 2], None, bins, [0, 180, 0, 256, 0, 256])
    listHist.append(hist)


In [ ]:

outArrayDp = np.asarray(listDp)
arrHist = np.array(listHist)
arrFFT = np.array([listFFT])
arrFFT = arrFFT.squeeze()
arrFFT = arrFFT.reshape((arrFFT.shape[0], arrFFT.shape[1] * arrFFT.shape[2] * arrFFT.shape[3]))
print(arrFFT.shape)
nsamples, nh, ns, nv = arrHist.shape
arrHist = arrHist.reshape(nsamples, nh * ns * nv)
print(arrHist.shape)
outArray = np.column_stack((arrFFT, arrHist))
print(outArray.shape)

In [ ]:
# model = KMeans(n_clusters=4)
# model.fit(arrFFT)
# all_predictions = model.predict(arrFFT)
# print(all_predictions)

In [ ]:
clust = OPTICS(min_samples=50, xi=0.05, min_cluster_size=0.05)

# Run the fit
clust.fit(outArray)
labels_050 = cluster_optics_dbscan(
    reachability=clust.reachability_,
    core_distances=clust.core_distances_,
    ordering=clust.ordering_,
    eps=0.5,
)
labels_200 = cluster_optics_dbscan(
    reachability=clust.reachability_,
    core_distances=clust.core_distances_,
    ordering=clust.ordering_,
    eps=2,
)

space = np.arange(len(outArray))
reachability = clust.reachability_[clust.ordering_]
labels = clust.labels_[clust.ordering_]

plt.figure(figsize=(10, 7))
G = gridspec.GridSpec(2, 3)
ax1 = plt.subplot(G[0, :])
ax2 = plt.subplot(G[1, 0])
ax3 = plt.subplot(G[1, 1])
ax4 = plt.subplot(G[1, 2])

# Reachability plot
colors = ["g.", "r.", "b.", "y.", "c."]
for klass, color in zip(range(0, 5), colors):
    Xk = space[labels == klass]
    Rk = reachability[labels == klass]
    ax1.plot(Xk, Rk, color, alpha=0.3)
ax1.plot(space[labels == -1], reachability[labels == -1], "k.", alpha=0.3)
ax1.plot(space, np.full_like(space, 2.0, dtype=float), "k-", alpha=0.5)
ax1.plot(space, np.full_like(space, 0.5, dtype=float), "k-.", alpha=0.5)
ax1.set_ylabel("Reachability (epsilon distance)")
ax1.set_title("Reachability Plot")

# OPTICS
colors = ["g.", "r.", "b.", "y.", "c."]
for klass, color in zip(range(0, 5), colors):
    Xk = outArray[clust.labels_ == klass]
    ax2.plot(Xk[:, 0], Xk[:, 1], color, alpha=0.3)
ax2.plot(outArray[clust.labels_ == -1, 0], outArray[clust.labels_ == -1, 1], "k+", alpha=0.1)
ax2.set_title("Automatic Clustering\nOPTICS")

# DBSCAN at 0.5
colors = ["g", "greenyellow", "olive", "r", "b", "c"]
for klass, color in zip(range(0, 6), colors):
    Xk = outArray[labels_050 == klass]
    ax3.plot(Xk[:, 0], Xk[:, 1], color, alpha=0.3, marker=".")
ax3.plot(outArray[labels_050 == -1, 0], outArray[labels_050 == -1, 1], "k+", alpha=0.1)
ax3.set_title("Clustering at 0.5 epsilon cut\nDBSCAN")

# DBSCAN at 2.
colors = ["g.", "m.", "y.", "c."]
for klass, color in zip(range(0, 4), colors):
    Xk = outArray[labels_200 == klass]
    ax4.plot(Xk[:, 0], Xk[:, 1], color, alpha=0.3)
ax4.plot(outArray[labels_200 == -1, 0], outArray[labels_200 == -1, 1], "k+", alpha=0.1)
ax4.set_title("Clustering at 2.0 epsilon cut\nDBSCAN")

plt.tight_layout()
plt.show()

In [ ]:
# Определяем модель и скорость обучения
model = TSNE(learning_rate=100)

# Обучаем модель
transformed = model.fit_transform(outArray)

# Представляем результат в двумерных координатах
x_axis = transformed[:, 0]
y_axis = transformed[:, 1]
plt.scatter(x_axis, y_axis)
plt.show()